# Stacked AutoEncoder

Importing the libraries

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

Importing the datasets

In [3]:
movies = pd.read_csv("ml-1m/movies.dat",sep = '::',header=None, engine='python',encoding='latin-1')
users = pd.read_csv("ml-1m/users.dat",sep = '::',header=None, engine='python',encoding='latin-1')
ratings = pd.read_csv("ml-1m/ratings.dat",sep = '::',header=None, engine='python',encoding='latin-1')

Preparing the training set and test set

In [4]:
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t')
training_set = np.array(training_set, dtype='int')
print(training_set)

[[        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]


In [5]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')
print(test_set)

[[        1        10         3 875693118]
 [        1        12         5 878542960]
 [        1        14         5 874965706]
 ...
 [      459       934         3 879563639]
 [      460        10         3 882912371]
 [      462       682         5 886365231]]


Getting the number of users and movies

In [6]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

Converting the data into an array with users in lines and movies in columns

In [7]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

Convert the data into Torch tensors

In [8]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Creating the architecture of the Neural Network

In [9]:
class SAE(nn.Module):
    def __init__(self,):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20,10)
        self.fc3 = nn.Linear(10,20)
        self.fc4 = nn.Linear(20,nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [10]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

Training the SAE

In [12]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0 :
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item() * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.7658592112372924
epoch: 2 loss: 1.0964134637180143
epoch: 3 loss: 1.0536830584773265
epoch: 4 loss: 1.0382909949295196
epoch: 5 loss: 1.0310620044676098
epoch: 6 loss: 1.0267053320579758
epoch: 7 loss: 1.0239449587877694
epoch: 8 loss: 1.0219318056099929
epoch: 9 loss: 1.0210546313072988
epoch: 10 loss: 1.0195844621216035
epoch: 11 loss: 1.0189739426598676
epoch: 12 loss: 1.0185247587317685
epoch: 13 loss: 1.018018790616514
epoch: 14 loss: 1.0176168652752278
epoch: 15 loss: 1.0171217448893952
epoch: 16 loss: 1.01705456736491
epoch: 17 loss: 1.016723847514434
epoch: 18 loss: 1.01673183166667
epoch: 19 loss: 1.0163626250834072
epoch: 20 loss: 1.0161124697718142
epoch: 21 loss: 1.0159719941030019
epoch: 22 loss: 1.0160097062376812
epoch: 23 loss: 1.0157539387107013
epoch: 24 loss: 1.0160778635089203
epoch: 25 loss: 1.0154315955196054
epoch: 26 loss: 1.0156884457278912
epoch: 27 loss: 1.0152938983309154
epoch: 28 loss: 1.0151347331629068
epoch: 29 loss: 1.0128958788173341


Testing the SAE

In [16]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item() * mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.9548043652479968
